## Model Building

##### Predicting wethere the water quality is safe for drinking or not by building ML models such as Logistic regression, Support vector Machine(SVM) and Decision tree along with hyperparameter tuning to predict the chance of diabetes. Using Recall as a performance metric to judge our models.

##### Choosing Recall as a performance metric helps to prioritise FALSE NEGATIVES where the model predicts the water is safe to drink but in reality it isn't, thus, creating a higher chance of risking lives.

### Lodaing the libraries 

In [1]:
import numpy as np
import pandas as pd
import warnings

from sklearn import datasets
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor

from imblearn.over_sampling import RandomOverSampler

from xgboost import XGBClassifier

from matplotlib import pyplot as plt

# Set random seed
np.random.seed(1)

# Ignore warnings
warnings.filterwarnings("ignore")

### Loading the processed training and test datasets

In [2]:
X_train = pd.read_csv('/Users/shambhavimishra/Downloads/X_train1.csv') 
y_train = pd.read_csv('/Users/shambhavimishra/Downloads/y_train1.csv') 
X_test = pd.read_csv('/Users/shambhavimishra/Downloads/X_test1.csv') 
y_test = pd.read_csv('/Users/shambhavimishra/Downloads/y_test1.csv') 

In [3]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(898, 19)
(898, 1)
(300, 19)
(300, 1)


### Building a dataframe to store our models performance metrics

In [4]:
performance = pd.DataFrame({"model": [], "Accuracy": [], "Precision": [], "Recall": [], "F1": []})

### Implementing Logistic Regression model with hyperparameters

##### Using Randomized search to find the best parameters

In [5]:
param_grid = {'penalty': ['l1', 'l2'], 
              'C': [0.001, 0.01, 0.1, 1, 10, 100],
              'solver': ['liblinear', 'saga'],
              'l1_ratio': [0.25, 0.5, 0.75],
             'max_iter': np.arange(800, 1200)
             }

# Perform Randomized Search CV to find the best hyperparameters
best_log_reg = RandomizedSearchCV(estimator=LogisticRegression(random_state=0, solver='saga'),
                                      scoring='recall', 
                                      param_distributions=param_grid, 
                                      cv=5, 
                                      verbose=0, 
                                      return_train_score=True, 
                                      n_iter=30, 
                                      n_jobs=-1)
best_log_reg.fit(X_train, y_train)

# Print the best parameters found through Randomized Search CV
print(f"Best parameters found through Randomized Search CV: {best_log_reg.best_params_}")


Best parameters found through Randomized Search CV: {'solver': 'saga', 'penalty': 'l1', 'max_iter': 1122, 'l1_ratio': 0.5, 'C': 1}


##### Performing GridSearch over a close range of parameters that we got from Randomized search to find the best parameters

In [6]:
# Define the parameter grid for Grid Search CV
param_grid = { 
    'solver': ['saga'],
    'penalty': ['l2'],
    'C': [0.1, 1, 10],
    'max_iter': np.arange(750,950)
}

# Perform Grid Search CV with the best parameters from Randomized Search CV
grid_log_reg = GridSearchCV(estimator=LogisticRegression(random_state=0, solver=best_log_reg.best_params_['solver']),
                                param_grid=param_grid,
                                scoring='recall',
                                cv=5,
                                n_jobs=-1)
grid_log_reg.fit(X_train, y_train)

# Print the best parameters found through Grid Search CV
print(f"Best parameters found through Grid Search CV: {grid_log_reg.best_params_}")

Best parameters found through Grid Search CV: {'C': 1, 'max_iter': 750, 'penalty': 'l2', 'solver': 'saga'}


##### Storing the performance metrics in the dataframe

In [7]:
# Define the parameter grid for Grid Search CV
param_grid = { 
    'solver': [best_log_reg.best_params_['solver']],
    'penalty': [best_log_reg.best_params_['penalty']],
    'C': [0.1, 1, 10],
    'max_iter': np.arange(750,950)
}

# Perform Grid Search CV with the best parameters from Randomized Search CV
grid_lregression = GridSearchCV(estimator=LogisticRegression(random_state=0, solver=best_log_reg.best_params_['solver']),
                                param_grid=param_grid,
                                scoring='recall',
                                cv=10,
                                n_jobs=-1)
grid_lregression.fit(X_train, y_train)

# Print the best parameters found through Grid Search CV
print(f"Best parameters found through Grid Search CV: {grid_lregression.best_params_}")

Best parameters found through Grid Search CV: {'C': 0.1, 'max_iter': 750, 'penalty': 'l1', 'solver': 'saga'}


In [8]:
# Evaluate the model using the best parameters found through Grid Search CV 
c_matrix = confusion_matrix(y_test, grid_lregression.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model': "LR", 
                                                     'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                     'Precision': [TP/(TP+FP)], 
                                                     'Recall': [TP/(TP+FN)], 
                                                     'F1': [2*TP/(2*TP+FP+FN)]
                                                    }, index=[0])])

In [9]:
performance

,model,Accuracy,Precision,Recall,F1
0,LR,0.883333,0.956284,0.866337,0.909091


### Implementing SVM model with hyperparameters

##### Using Randomized search to find the best parameters

In [10]:
score_measure = "recall"
kfolds = 5

param_grid = {
    'C': np.arange(1,25),   
    'gamma': ['scale','auto'],
    'kernel':['linear','rbf','poly']
}

svm = SVC()
rand_search = RandomizedSearchCV(estimator = svm, param_distributions=param_grid, cv=kfolds, n_iter=140,
                           scoring=score_measure, verbose=1, n_jobs=-1, 
                           return_train_score=True)

_ = rand_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {rand_search.best_score_}")
print(f"... with parameters: {rand_search.best_params_}")

Fitting 5 folds for each of 140 candidates, totalling 700 fits
The best recall score is 0.8530836454431959
... with parameters: {'kernel': 'poly', 'gamma': 'auto', 'C': 2}


##### Performing GridSearch over a close range of parameters that we got from Randomized search to find the best parameters

In [11]:
score_measure = "recall"
kfolds = 5

C = rand_search.best_params_['C']
gamma = rand_search.best_params_['gamma']
kernel = rand_search.best_params_['kernel']

param_grid = {
    'C': np.arange(C-2,C+2),  
    'gamma': [gamma],
    'kernel': [kernel]
    
}

svm1 = SVC()
grid_search = GridSearchCV(estimator = svm1, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1, # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = grid_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {grid_search.best_score_}")
print(f"... with parameters: {grid_search.best_params_}")

bestprecision_SVM = grid_search.best_estimator_

Fitting 5 folds for each of 4 candidates, totalling 20 fits
The best recall score is 0.8530836454431959
... with parameters: {'C': 2, 'gamma': 'auto', 'kernel': 'poly'}


##### Storing the performance metrics in the dataframe

In [12]:
c_matrix = confusion_matrix(y_test, grid_search.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"SVM", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,LR,0.883333,0.956284,0.866337,0.909091
0,SVM,0.793333,0.872340,0.811881,0.841026


### Implementing Decision Tree model with hyperparameters

##### Using Randomized search to find the best parameters

In [13]:
score_measure = "recall"
kfolds = 5

param_grid = {
    'min_samples_split': np.arange(2,50),  
    'min_samples_leaf': np.arange(1,50),
    'min_impurity_decrease': np.arange(0.0001, 0.01, 0.0005),
    'max_leaf_nodes': np.arange(5, 50), 
    'max_depth': np.arange(1,20), 
    'criterion': ['gini', 'entropy'],
}

dtree = DecisionTreeClassifier()
rand_search = RandomizedSearchCV(estimator=dtree, param_distributions=param_grid, cv=kfolds, n_iter=500,
                                 scoring=score_measure, verbose=1, n_jobs=-1, # n_jobs=-1 will utilize all available CPUs 
                                 return_train_score=True)

_ = rand_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {rand_search.best_score_}")
print(f"... with parameters: {rand_search.best_params_}")

bestPrecTree = rand_search.best_estimator_

Fitting 5 folds for each of 500 candidates, totalling 2500 fits
The best recall score is 0.8485892634207242
... with parameters: {'min_samples_split': 36, 'min_samples_leaf': 17, 'min_impurity_decrease': 0.0091, 'max_leaf_nodes': 31, 'max_depth': 14, 'criterion': 'entropy'}


##### Performing GridSearch over a close range of parameters that we got from Randomized search to find the best parameters

In [14]:
score_measure = "recall"
kfolds = 5

param_grid = {
    'min_samples_split': np.arange(26,36),  
    'min_samples_leaf': np.arange(8,16),
    'min_impurity_decrease': np.arange( 0.0005, 0.0010, 0.0020),
    'max_leaf_nodes': [10,30], 
    'max_depth': [5,15], 
    'criterion': ['entropy']
}


dtree = DecisionTreeClassifier()
grid_search = GridSearchCV(estimator = dtree, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = grid_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {grid_search.best_score_}")
print(f"... with parameters: {grid_search.best_params_}")

bestPrecisionTree = grid_search.best_estimator_

Fitting 5 folds for each of 320 candidates, totalling 1600 fits
The best recall score is 0.8397003745318352
... with parameters: {'criterion': 'entropy', 'max_depth': 15, 'max_leaf_nodes': 30, 'min_impurity_decrease': 0.0005, 'min_samples_leaf': 13, 'min_samples_split': 33}


##### Storing the performance metrics in the dataframe

In [15]:
c_matrix = confusion_matrix(y_test, grid_search.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"Decision Tree", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,LR,0.883333,0.956284,0.866337,0.909091
0,SVM,0.793333,0.872340,0.811881,0.841026
0,Decision Tree,0.806667,0.887097,0.816832,0.850515


### Implementing Neural Network model with hyperparameters

In [16]:
%%time

ann = MLPClassifier(hidden_layer_sizes=(60,50,40), solver='adam', max_iter=200) #max_iter - how often we are changing the w's(weight)
_ = ann.fit(X_train, y_train)

CPU times: total: 3.34 s
Wall time: 2.28 s


In [17]:
%%time
y_pred = ann.predict(X_test)

CPU times: total: 0 ns
Wall time: 0 ns


In [18]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.73      0.73      0.73        98
           1       0.87      0.87      0.87       202

    accuracy                           0.83       300
   macro avg       0.80      0.80      0.80       300
weighted avg       0.83      0.83      0.83       300



##### NN model with RandomSearchCV

In [19]:
%%time

score_measure = "accuracy"
kfolds = 5

param_grid = {
    'hidden_layer_sizes': [ (50,), (70,),(50,30), (40,20), (60,40, 20), (70,50,40)],
    'activation': ['logistic', 'tanh', 'relu'],
    'solver': ['adam', 'sgd'],
    'alpha': [0, .2, .5, .7, 1],     #regulization term
    'learning_rate': ['constant', 'invscaling', 'adaptive'],
    'learning_rate_init': [0.001, 0.01, 0.1, 0.2, 0.5],
    'max_iter': [5000]
}

ann = MLPClassifier()
grid_search = RandomizedSearchCV(estimator = ann, param_distributions=param_grid, cv=kfolds, n_iter=100,
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = grid_search.fit(X_train, y_train)

bestRecallTree = grid_search.best_estimator_

print(grid_search.best_params_)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
{'solver': 'adam', 'max_iter': 5000, 'learning_rate_init': 0.01, 'learning_rate': 'adaptive', 'hidden_layer_sizes': (70,), 'alpha': 0.2, 'activation': 'tanh'}
CPU times: total: 3.61 s
Wall time: 1min 29s


In [20]:
%%time
y_pred = bestRecallTree.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.75      0.80      0.77        98
           1       0.90      0.87      0.88       202

    accuracy                           0.85       300
   macro avg       0.82      0.83      0.83       300
weighted avg       0.85      0.85      0.85       300

CPU times: total: 15.6 ms
Wall time: 16.6 ms


##### NN model with GridSearchCV

In [21]:
%%time

score_measure = "accuracy"
kfolds = 5

param_grid = {
    'hidden_layer_sizes': [ (30,), (50,), (70,), (90,)],
    'activation': ['tanh', 'relu'],
    'solver': ['adam'],
    'alpha': [.5, .7, 1],
    'learning_rate': ['adaptive', 'invscaling'],
    'learning_rate_init': [0.005, 0.01, 0.15],
    'max_iter': [5000]
}

ann = MLPClassifier()
grid_search = GridSearchCV(estimator = ann, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = grid_search.fit(X_train, y_train)

bestRecallTree = grid_search.best_estimator_

print(grid_search.best_params_)

Fitting 5 folds for each of 144 candidates, totalling 720 fits
{'activation': 'tanh', 'alpha': 0.5, 'hidden_layer_sizes': (90,), 'learning_rate': 'invscaling', 'learning_rate_init': 0.01, 'max_iter': 5000, 'solver': 'adam'}
CPU times: total: 4.44 s
Wall time: 55.8 s


In [22]:
%%time
y_pred = bestRecallTree.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.77      0.82      0.79        98
           1       0.91      0.88      0.89       202

    accuracy                           0.86       300
   macro avg       0.84      0.85      0.84       300
weighted avg       0.86      0.86      0.86       300

CPU times: total: 62.5 ms
Wall time: 15.6 ms


In [23]:
c_matrix = confusion_matrix(y_test, grid_search.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"Neural Network", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,LR,0.883333,0.956284,0.866337,0.909091
0,SVM,0.793333,0.872340,0.811881,0.841026
0,Decision Tree,0.806667,0.887097,0.816832,0.850515
0,Neural Network,0.860000,0.908163,0.881188,0.894472


# Using Keras

### Deep Neural Network

In [33]:
import tensorflow as tf
from tensorflow import keras

# fix random seed for reproducibility
np.random.seed(1)
tf.random.set_seed(1)

# create model stucture
model = keras.models.Sequential()
model.add(keras.layers.Input(shape=(19,))) # update input shape
model.add(keras.layers.Dense(50, activation='relu', kernel_initializer=tf.keras.initializers.GlorotNormal()))
model.add(keras.layers.Dense(50, activation='relu', kernel_initializer=tf.keras.initializers.GlorotNormal()))
model.add(keras.layers.Dense(50, activation='relu', kernel_initializer=tf.keras.initializers.GlorotNormal()))
model.add(keras.layers.Dense(10, activation='softmax', kernel_initializer=tf.keras.initializers.GlorotNormal())) # final layer, 10 categories

# compile
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# if you want to overide the defaults for the optimizer....
#adam = keras.optimizers.Adam(learning_rate=0.01)
#model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
# fit the model
history = model.fit(X_train, y_train, 
                    validation_data=(X_test, y_test), 
                    epochs=20, batch_size=100)


Epoch 1/20
9/9 [==============================] - 1s 26ms/step - loss: 2.2005 - accuracy: 0.3519 - val_loss: 1.9018 - val_accuracy: 0.6600
Epoch 2/20
9/9 [==============================] - 0s 4ms/step - loss: 1.8344 - accuracy: 0.5067 - val_loss: 1.5163 - val_accuracy: 0.6800
Epoch 3/20
9/9 [==============================] - 0s 4ms/step - loss: 1.4253 - accuracy: 0.5356 - val_loss: 1.0730 - val_accuracy: 0.6867
Epoch 4/20
9/9 [==============================] - 0s 4ms/step - loss: 0.9837 - accuracy: 0.6459 - val_loss: 0.7338 - val_accuracy: 0.7500
Epoch 5/20
9/9 [==============================] - 0s 4ms/step - loss: 0.6724 - accuracy: 0.7294 - val_loss: 0.5920 - val_accuracy: 0.7200
Epoch 6/20
9/9 [==============================] - 0s 4ms/step - loss: 0.5343 - accuracy: 0.7673 - val_loss: 0.4799 - val_accuracy: 0.7733
Epoch 7/20
9/9 [==============================] - 0s 5ms/step - loss: 0.4593 - accuracy: 0.7951 - val_loss: 0.4336 - val_accuracy: 0.8033
Epoch 8/20
9/9 [=================

In [34]:
# evaluate the model

scores = model.evaluate(X_test, y_test, verbose=0)
scores
# In results, first is loss, second is accuracy

[0.34170570969581604, 0.8666666746139526]

In [35]:
# let's format this into a better output...

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

loss: 0.34
accuracy: 86.67%


### Wide and Deep Neural Network

In [36]:
import tensorflow as tf
from tensorflow import keras

# Define the model: for multi-class
model = keras.models.Sequential()
model.add(keras.layers.Input(shape=(19,))) # update input shape
model.add(keras.layers.Dense(100, activation='relu', kernel_initializer='glorot_normal'))
model.add(keras.layers.Dense(100, activation='relu', kernel_initializer='glorot_normal'))
model.add(keras.layers.Dense(100, activation='relu', kernel_initializer='glorot_normal'))
model.add(keras.layers.Dense(10, activation='softmax', kernel_initializer='glorot_normal'))

# Compile model
adam = keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])


In [37]:
# Fit the model
history = model.fit(X_train, y_train, 
                    validation_data=(X_test, y_test), 
                    epochs=20, batch_size=100)

Epoch 1/20
9/9 [==============================] - 1s 23ms/step - loss: 0.8678 - accuracy: 0.6481 - val_loss: 0.3443 - val_accuracy: 0.8367
Epoch 2/20
9/9 [==============================] - 0s 5ms/step - loss: 0.3385 - accuracy: 0.8597 - val_loss: 0.3134 - val_accuracy: 0.8633
Epoch 3/20
9/9 [==============================] - 0s 5ms/step - loss: 0.2860 - accuracy: 0.8675 - val_loss: 0.4026 - val_accuracy: 0.8500
Epoch 4/20
9/9 [==============================] - 0s 5ms/step - loss: 0.2286 - accuracy: 0.9076 - val_loss: 0.4021 - val_accuracy: 0.8433
Epoch 5/20
9/9 [==============================] - 0s 5ms/step - loss: 0.1867 - accuracy: 0.9187 - val_loss: 0.3605 - val_accuracy: 0.8533
Epoch 6/20
9/9 [==============================] - 0s 5ms/step - loss: 0.1456 - accuracy: 0.9443 - val_loss: 0.4743 - val_accuracy: 0.8267
Epoch 7/20
9/9 [==============================] - 0s 5ms/step - loss: 0.1143 - accuracy: 0.9633 - val_loss: 0.5296 - val_accuracy: 0.8267
Epoch 8/20
9/9 [=================

In [38]:
# Evaluate the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Loss: %.2f" % scores[0])
print("Accuracy: %.2f%%" % (scores[1]*100))

Loss: 0.69
Accuracy: 81.00%


## RandomGridSearch

In [47]:
%%time

# If you don't have the following installed, from command line '!pip install scikeras'
from scikeras.wrappers import KerasClassifier
from keras.initializers import GlorotNormal

score_measure = "accuracy"
kfolds = 5

def build_clf(hidden_layer_sizes, dropout):
    ann = tf.keras.models.Sequential()
    ann.add(keras.layers.Input(shape=19)),
    for hidden_layer_size in hidden_layer_sizes:
        model.add(keras.layers.Dense(hidden_layer_size, kernel_initializer= tf.keras.initializers.GlorotUniform(), 
                                     bias_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None), activation="relu"))
        model.add(keras.layers.Dropout(dropout))
    ann.add(tf.keras.layers.Dense(10, activation='softmax'))
    ann.compile(loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
    return ann


CPU times: total: 0 ns
Wall time: 0 ns


In [48]:
from scikeras.wrappers import KerasClassifier

keras_clf = KerasClassifier(
    model=build_clf,
    hidden_layer_sizes=19,
    dropout = 0.0
)

In [49]:
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import RandomizedSearchCV

params = {
    'optimizer__learning_rate': [0.0005, 0.001, 0.005],
    'model__hidden_layer_sizes': [(70,),(90, ), (100,), (100, 90)],
    'model__dropout': [0, 0.1],
    'batch_size':[20, 60, 100],
    'epochs':[10, 50, 100],
    'optimizer':["adam",'sgd']
}
keras_clf.get_params().keys()

dict_keys(['model', 'build_fn', 'warm_start', 'random_state', 'optimizer', 'loss', 'metrics', 'batch_size', 'validation_batch_size', 'verbose', 'callbacks', 'validation_split', 'shuffle', 'run_eagerly', 'epochs', 'hidden_layer_sizes', 'dropout', 'class_weight'])

In [50]:
rnd_search_cv = RandomizedSearchCV(estimator=keras_clf, param_distributions=params, scoring='accuracy', n_iter=50, cv=5)

import sys
sys.setrecursionlimit(10000) # note: the default is 3000 (python 3.9)

earlystop = EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='auto')
callback = [earlystop]

_ = rnd_search_cv.fit(X_train, y_train, callbacks=callback, verbose=0)

3/3 [==============================] - 0s 4ms/step


In [51]:
rnd_search_cv.best_params_

{'optimizer__learning_rate': 0.0005,
 'optimizer': 'sgd',
 'model__hidden_layer_sizes': (100,),
 'model__dropout': 0.1,
 'epochs': 100,
 'batch_size': 60}

In [52]:
best_net = rnd_search_cv.best_estimator_
print(rnd_search_cv.best_params_)

{'optimizer__learning_rate': 0.0005, 'optimizer': 'sgd', 'model__hidden_layer_sizes': (100,), 'model__dropout': 0.1, 'epochs': 100, 'batch_size': 60}


In [53]:
%%time
y_pred = best_net.predict(X_test)
print(classification_report(y_test, y_pred))

5/5 [==============================] - 0s 1ms/step
              precision    recall  f1-score   support

           0       0.71      0.94      0.81        98
           1       0.96      0.82      0.88       202

    accuracy                           0.86       300
   macro avg       0.84      0.88      0.85       300
weighted avg       0.88      0.86      0.86       300

CPU times: total: 78.1 ms
Wall time: 87.9 ms


## Conclusion:

#### The models used: Logistic regression with hyperparameter tuning, SVM model with hyperparameter tuning, Decision Tree with hyperparameter tuning, Neural Network with randomsearchcv and gridsearchcv, Deep Network, Wide and Deep Neural Network with randomsearchcv.

Looking at the performance metric, the hightest Accuracy is of the Linear Regression Model(88.3%), then Wide & Deep Neural Network (86.67%), Neural Network (84.3%), followed by Decision Tree(80.6%) and SVM(84.3%).

Looking at the performance metric, the hightest Precision is of the Wide & Deep Neural Network (96%), then Linear Regression Model(95.6%), then Neural Network(90.1%), followed by Decision Tree(88.7%) and SVM(87.2%).

Looking at the performance metric, the hightest Recall is of the Linear Regression Model(86.6%), then Neural Network(86.1%), followed by Decision Tree(81.6%), SVM(81.1%) and Wide & Deep Neural Network(81%).

Looking at the performance metric, the hightest F1 is of the Linear Regression Model(90.9%), then Neural Network(88.1%), Wide & Deep Neural Network(88%), followed by Decision Tree(85.1%) and SVM(84.1%).

##### Thus, it is evident that Linear Regression Model is the best model for predicting whether the water is safe or not as it has an the best performance matric scores(Accuracy, Precision, Recall, F1) compared to the SVM, Decision Tree and Neural Network Model, Deep Neural Network and Wide & Deep Neural Network.
